In [1]:
import tensorflow as tf

In [2]:
x_data = [[1,2,1,1],[2,1,3,2],[3,1,3,4],[4,1,5,5],[1,7,5,5],[1,2,5,6],[1,6,6,6],[1,7,7,7]]
y_data = [[0,0,1],[0,0,1],[0,0,1],[0,1,0],[0,1,0],[0,1,0],[1,0,0],[1,0,0]]
# y_data는 'One-Hot' Encoding으로 표시됨
# [0,0,1] = 2, [0,1,0] = 1, [1,0,0] = 0

In [4]:
X = tf.placeholder("float",[None,4])
Y = tf.placeholder("float",[None,3])
nb_classes = 3
# label의 갯수 (0,1,2)

In [5]:
W = tf.Variable(tf.random_normal([4,nb_classes]),name='weight')
b = tf.Variable(tf.random_normal([nb_classes]),name='bias')

In [6]:
# tf.nn.softmax computes softmax activations
# Softmax = exp(logits) / reduce_sum(exp(logits),dim)
hypothesis = tf.nn.softmax(tf.matmul(X,W)+b)

In [8]:
# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis),axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [9]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={X:x_data,Y:y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))

0 6.50811
200 0.659538
400 0.565202
600 0.47929
800 0.393049
1000 0.304907
1200 0.229045
1400 0.207159
1600 0.189775
1800 0.174991
2000 0.162275


학습 끝!

---
그럼 데이터를 테스트할 차례

# Test & One-Hot encoding

``` 
hypothesis = tf.nn.softmax(tf.matmul(X,W)+b)
```

In [14]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={X:x_data,Y:y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))

    # Testing & One-hot encoding
    a = sess.run(hypothesis, feed_dict={X: [[1,11,7,9]]})
    print(a,sess.run(tf.arg_max(a,1)))
    
    print('-------------------')
    
    b = sess.run(hypothesis, feed_dict={X:[[1,3,4,3]]})
    print(b, sess.run(tf.arg_max(b,1)))
    
    print('-------------------')
    
    c = sess.run(hypothesis, feed_dict = {X:[[1,1,0,1]]})
    print (c,sess.run(tf.arg_max(c,1)))
    
    print('-------------------')
    
    all = sess.run(hypothesis, feed_dict = {X:[[1,11,7,9],[1,3,4,3],[1,1,0,1]]})
    print(all, sess.run(tf.arg_max(all,1)))

0 6.93039
200 0.689629
400 0.568088
600 0.482945
800 0.410418
1000 0.342119
1200 0.271706
1400 0.230495
1600 0.210104
1800 0.192857
2000 0.178098
Instructions for updating:
Use `argmax` instead
[[  2.13666797e-01   7.86323071e-01   1.01224196e-05]] [1]
-------------------
Instructions for updating:
Use `argmax` instead
[[ 0.38355446  0.55169487  0.0647507 ]] [1]
-------------------
Instructions for updating:
Use `argmax` instead
[[  3.26209566e-08   5.13924111e-04   9.99485970e-01]] [2]
-------------------
Instructions for updating:
Use `argmax` instead
[[  2.13666797e-01   7.86323071e-01   1.01224205e-05]
 [  3.83554459e-01   5.51694870e-01   6.47507012e-02]
 [  3.26209566e-08   5.13924111e-04   9.99485970e-01]] [1 1 2]


마지막 결과 값인 [x,y,z]은 [0,1,2] 중에 나올텐데<br>
첫째줄에 [x1,y1,z1] 중 가장 큰값(argmax)의 위치. e.g. x1이 제일 크면 [0]<br>
[x1,y1,z1]<br>
[x2,y2,z2]<br>
[x3,y3,z3]

# 6-2 Fancy Softmax Classifier
## cross_entropy, one_hot, reshape

============================================
## softmax_cross_entropy_with_logits

```
logits = tf.matmul(X,W)+b
hypothesis = tf.nn.softmax(logits)
```

In [18]:
logits = tf.matmul(X,W)+b
hypothesis=tf.nn.softmax(logits)

In [19]:
# 1. Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis),axis = 1))

In [21]:
# 2. Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                labels=Y)
cost = tf.reduce_mean(cost_i)

=========================================================

# Animal classification
#### with softmax_cross_entropy_with_logits

In [22]:
import numpy as np

In [24]:
xy=np.loadtxt('data-04-zoo.csv',delimiter=',',dtype=np.float32)
x_data = xy[:,0:-1]
y_data = xy[:,[-1]]
nb_classes=7

데이터를 살펴보면 Y는 하나

In [25]:
Y = tf.placeholder(tf.int32,[None,1]) # 0~6, shape(?,1)
Y_one_hot = tf.one_hot(Y,nb_classes)  # One hot shape(?,1,7)
Y_one_hot = tf.reshape(Y_one_hot,[-1,nb_classes]) # shape(?,7)

If the input **indices** is rank N, the output will have rank **N+1**.<br>
The new axis is created at dimension axis (default: the new axis is appended at the end)

In [26]:
# one hot을 하면 한차원이 더 더해진다.
# [[0],[3]] -> [[[1000000]],[[0001000]]]
# rank=2였던게 r=3이 되어버림
# 그러므로 reshape이 필요
# Y_one_hot = tf.reshape(Y_one_hot,[-1,nb_classes]) 여기서 -1은 나머지(=알아서)

=====================================================================

# 한번에

In [27]:
# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv',delimiter=',',dtype=np.float32)
x_data = xy[:,0:-1]
y_data = xy[:,[-1]]
nb_classes = 7 # 0~6

In [30]:
X = tf.placeholder(tf.float32,[None, 16])
Y = tf.placeholder(tf.int32,[None, 1]) # 0~6

In [31]:
Y_one_hot = tf.one_hot(Y,nb_classes)  # one hot
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

In [32]:
W = tf.Variable(tf.random_normal([16,nb_classes]),name='weight')
b = tf.Variable(tf.random_normal([nb_classes]),name='bias')

In [33]:
# tf.nn.softmax computes softmax activations
# sofmax = exp(logits) / reduce_sum(exp(logits),dim)
logits = tf.matmul(X,W)+b
hypothesis = tf.nn.softmax(logits)

In [34]:
# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits,
                                                labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

In [36]:
# 예측한 값이 맞나 확인해보고 싶음
prediction = tf.argmax(hypothesis,1) # probabilities를 0~6의 값으로 만들어줌
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot,1)) #(label, Y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [37]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range (2000):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost,accuracy],
                                feed_dict={X: x_data, Y:y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step,loss,acc))
            
    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X:x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p,y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p ,int(y)))

Step:     0	Loss: 5.931	Acc: 4.95%
Step:   100	Loss: 0.728	Acc: 77.23%
Step:   200	Loss: 0.445	Acc: 90.10%
Step:   300	Loss: 0.329	Acc: 90.10%
Step:   400	Loss: 0.260	Acc: 92.08%
Step:   500	Loss: 0.215	Acc: 93.07%
Step:   600	Loss: 0.183	Acc: 98.02%
Step:   700	Loss: 0.160	Acc: 98.02%
Step:   800	Loss: 0.142	Acc: 98.02%
Step:   900	Loss: 0.128	Acc: 98.02%
Step:  1000	Loss: 0.116	Acc: 98.02%
Step:  1100	Loss: 0.106	Acc: 98.02%
Step:  1200	Loss: 0.098	Acc: 98.02%
Step:  1300	Loss: 0.091	Acc: 99.01%
Step:  1400	Loss: 0.085	Acc: 99.01%
Step:  1500	Loss: 0.079	Acc: 99.01%
Step:  1600	Loss: 0.074	Acc: 100.00%
Step:  1700	Loss: 0.070	Acc: 100.00%
Step:  1800	Loss: 0.066	Acc: 100.00%
Step:  1900	Loss: 0.063	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 True Y: 

data.flatten은 [[1],[0]] -> [1,0] <br>
zip은 리스트를 묶어서 각각의 p, y로 넘겨주기 용이하게 만듬